# Random Forest

Here we explore the usage of RF

In [ ]:
%matplotlib inline

import os
import binascii

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.metrics import roc_auc_score
from sklearn.tree import export_graphviz

For now we will be ok with 30 depth which is already too much.

$n \in [1, 90000]$

$d \in [1, 30]$

In [ ]:
ESTIMATOR_LEN = 500000
MAX_DEPTH = 4

estimator_range = range(1, 99001, 10)
depth_range = range(1, 31)

pd.options.display.max_rows = 4000
pd.options.display.max_columns = 4000

First we will load the data

In [ ]:
raw_all_features_df = pd.read_csv("training_set_features.csv", index_col="respondent_id")
all_labels_df = pd.read_csv("training_set_labels.csv", index_col="respondent_id")

test_set_features_df = pd.read_csv("test_set_features.csv", index_col="respondent_id")

submission_df = pd.read_csv("submission_format.csv", index_col="respondent_id")

raw_train_features_df, raw_test_features_df, train_labels_df, test_labels_df = train_test_split(
    raw_all_features_df,
    all_labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=all_labels_df,
    random_state=25519
)

raw_train_features_df

In [ ]:
def ordinalize(features: pd.DataFrame, raw: pd.DataFrame, name: str) -> pd.DataFrame:
    ordinal_arr = OrdinalEncoder().fit_transform(raw[[name]])
    
    features.drop(name, axis=1)
    features[name] = ordinal_arr
    return features

    
def onehotize(features: pd.DataFrame, raw: pd.DataFrame, name: str) -> pd.DataFrame:
    enc = OneHotEncoder(sparse=False)
    onehot_df = pd.DataFrame(
        enc.fit_transform(raw[name]),
        columns=enc.get_feature_names_out(),
    )
    onehot_df.index = raw.index
    return pd.concat(
        [features.drop(name, axis=1), onehot_df],
        axis=1,
    )

def normalize(in_df: pd.DataFrame) -> pd.DataFrame:
    out_df = in_df
    out_df = onehotize(out_df, in_df, ["employment_industry", "employment_occupation", "race", "employment_status", "hhs_geo_region", "census_msa"])
    out_df = ordinalize(out_df, in_df, "age_group")
    out_df = ordinalize(out_df, in_df, "education")
    out_df = ordinalize(out_df, in_df, "sex")
    out_df = ordinalize(out_df, in_df, "income_poverty")
    out_df = ordinalize(out_df, in_df, "marital_status")
    out_df = ordinalize(out_df, in_df, "rent_or_own")
    return out_df.fillna(out_df.mean())

train_features_df = normalize(raw_train_features_df)

test_features_df = normalize(raw_test_features_df)

train_features_df

In [ ]:
regress_multirf = MultiOutputRegressor(
    RandomForestRegressor(
        n_estimators=ESTIMATOR_LEN,
        max_depth=MAX_DEPTH,
        random_state=31415
    )
)

How we will fit:

In [ ]:
%%time

regress_multirf.fit(train_features_df, train_labels_df)

In [36]:
export_graphviz(
    regress_multirf.estimators_[0][25],
    out_file=f'tree_{str(binascii.hexlify(os.urandom(8)))}.dot', 
    feature_names = train_features_df.columns,
    class_names = train_labels_df.columns,
    rounded = True,
    proportion = False,
    precision = 2,
    filled = True    
)

In [82]:
preds = regress_multirf.predict(test_features_df)

preds_df = pd.DataFrame({"h1h1_vaccine": preds[:, 0], "seasonal_vaccine": preds[:, 1],}, index=test_labels_df.index)
preds_df

,h1h1_vaccine,seasonal_vaccine
respondent_id,,
26163,0.208333,0.241667
2855,0.025000,0.008333
927,0.175000,0.108333
13814,0.058333,0.083333
18594,0.175000,0.575000
...,...,...
22971,0.183333,0.324405
2491,0.133333,0.058333
14076,0.108333,0.632742


In [83]:
roc_auc_score(test_labels_df, preds_df)

0.8282981673273344

Now we have our tests, lets train with the whole dataset:

In [ ]:
features_submission_df = normalize(features_all_df)
features_submission_df

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,sex,income_poverty,marital_status,rent_or_own,employment_status,household_adults,household_children,employment_industry_arjwrbjb,employment_industry_atmlpfrs,employment_industry_cfqqtusy,employment_industry_dotnnunm,employment_industry_fcxhlnwr,employment_industry_haxffmxo,employment_industry_ldnlellj,employment_industry_mcubkhph,employment_industry_mfikgejo,employment_industry_msuufmds,employment_industry_nduyfdeo,employment_industry_phxvnwax,employment_industry_pxcmvdjn,employment_industry_qnlwzans,employment_industry_rucpziij,employment_industry_saaquncn,employment_industry_vjjrobsf,employment_industry_wlfvacwt,employment_industry_wxleyezf,employment_industry_xicduogh,employment_industry_xqicxuve,employment_industry_nan,employment_occupation_bxpfxfdn,employment_occupation_ccgxvspp,employment_occupation_cmhcxjea,employment_occupation_dcjcmpih,employment_occupation_dlvbwzss,employment_occupation_emcorrxb,employment_occupation_haliazsg,employment_occupation_hfxkjkmi,employment_occupation_hodpvpew,employment_occupation_kldqjyjy,employment_occupation_mxkfnird,employment_occupation_oijqvulv,employment_occupation_pvmttkik,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv,employment_occupation_nan,race_Black,race_Hispanic,race_Other or Multiple,race_White,hhs_geo_region_atmpeygn,hhs_geo_region_bhuqouqj,hhs_geo_region_dqpwygqj,hhs_geo_region_fpwskwrf,hhs_geo_region_kbazzjca,hhs_geo_region_lrircsnp,hhs_geo_region_lzgpxyit,hhs_geo_region_mlyzmhmf,hhs_geo_region_oxchjgsf,hhs_geo_region_qufhixun,"census_msa_MSA, Not Principle City","census_msa_MSA, Principle City",census_msa_Non-MSA
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,3.0,1.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,1.0,0.0,0.0,-1.0,3.0,1.0,1.0,4.0,1.0,2.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-1.0,3.0,3.0,5.0,5.0,4.0,1.0,4.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1

In [54]:
%%time

features_submission_df = normalize(features_all_df)

multirf = MultiOutputRegressor(
    RandomForestRegressor(
        n_estimators=ESTIMATOR_LEN,
        max_depth=MAX_DEPTH,
        random_state=31415
    )
)

multirf.fit(features_submission_df, labels_all_df)

predictions = multirf.predict(normalize(test_set_features_df))

predictions

CPU times: user 1min 26s, sys: 500 ms, total: 1min 27s
Wall time: 1min 27s


array([[0.19868375, 0.24227426],
       [0.03393042, 0.12230072],
       [0.16380432, 0.72899469],
       ...,
       [0.1072032 , 0.25725513],
       [0.03383633, 0.48236799],
       [0.70015074, 0.74455142]])

In [55]:
submission_name = f"submission_{str(binascii.hexlify(os.urandom(8)))}.csv"

np.testing.assert_array_equal(normalize(test_set_features_df).index.values,  submission_df.index.values)

submission_df["h1n1_vaccine"] = predictions[:, 0]
submission_df["seasonal_vaccine"] = predictions[:, 1]

submission_df.to_csv(submission_name, index=True)

submission_df.head()

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.198684,0.242274
26708,0.033930,0.122301
26709,0.163804,0.728995
26710,0.526216,0.878728
26711,0.358076,0.547638
